In [ ]:
import duckdb
cn = duckdb.connect()

import data_gen
data_gen.generate_bicycle_orders(cn, target_rows=50_000)


In [4]:

sql = """
with agg as (
    select order_date, sum(amount) as amount
    from bicycle_orders
    group by all
)
SELECT 
    order_date,
    amount,
    LAG(amount, 1) OVER (ORDER BY order_date) as prev_day_amount,
    amount - prev_day_amount as amount_change,
    AVG(amount) OVER (ORDER BY order_date ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) as rolling_3day_avg,
    SUM(amount) OVER (ORDER BY order_date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) as rolling_7day_sum
FROM agg
ORDER BY order_date
limit 10
"""

cn.sql(sql)

┌────────────┬───────────────┬─────────────────┬───────────────┬────────────────────┬──────────────────┐
│ order_date │    amount     │ prev_day_amount │ amount_change │  rolling_3day_avg  │ rolling_7day_sum │
│    date    │ decimal(38,2) │  decimal(38,2)  │ decimal(38,2) │       double       │  decimal(38,2)   │
├────────────┼───────────────┼─────────────────┼───────────────┼────────────────────┼──────────────────┤
│ 2024-01-01 │     276209.23 │            NULL │          NULL │          276209.23 │        276209.23 │
│ 2024-01-02 │     351490.41 │       276209.23 │      75281.18 │          313849.82 │        627699.64 │
│ 2024-01-03 │     355623.71 │       351490.41 │       4133.30 │          327774.45 │        983323.35 │
│ 2024-01-04 │     317801.30 │       355623.71 │     -37822.41 │  341638.4733333333 │       1301124.65 │
│ 2024-01-05 │     328590.53 │       317801.30 │      10789.23 │          334005.18 │       1629715.18 │
│ 2024-01-06 │     393924.69 │       328590.53 │      6